In [12]:
import trafilatura
import json
from langdetect import detect
import hanzidentifier
from urllib.parse import urlparse
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [13]:
def extract_YI_XIANG_HAO_JU_link():
    links = []
    for page_number in range(70):
        if page_number == 0:
            url = f"https://www.uhouzz.com/news/tags/aozhouzufang.html"
        else:
            url = f"https://www.uhouzz.com/news/tags/aozhouzufang_{page_number}.html"
        print(f'accessing {url}')
        resp = requests.get(url, headers={"User-Agent": "MyBot/1.0"})
        time.sleep(2)
        soup = BeautifulSoup(resp.text, "lxml")

        # Extract all href links
        for a in soup.find_all("a", href=True):
            full_url = urljoin(url, a["href"])  # make relative URLs absolute
            if full_url.startswith('https://www.uhouzz.com/news/'):
                links.append(full_url)
    return links

In [14]:
def load_web(url):
    print(f'start fetch {url}')
    # Download HTML
    downloaded = trafilatura.fetch_url(url)
    # Extract with metadata
    data_json = trafilatura.extract(
        downloaded,
        output_format="json",   # structured output
        with_metadata=True,     # include title, author, date, etc.
        include_comments=False,
        include_images=False
    )
    if data_json:
        data = json.loads(data_json)

        # return none for 404 page
        if data.get('title') == 'undefined':
            return None
        
        return data
    else:
        return None

In [15]:
def check_language(text):
    # detect language
    try:
        language = detect(text)
    except:
        return None

    # check chinese script type
    if language.startswith('zh'):
        has_simp = hanzidentifier.is_simplified(text)
        has_trad = hanzidentifier.is_traditional(text)
        
        if has_simp and not has_trad:
            return "simplified-chinese"
        elif has_trad and not has_simp:
            return "traditional-chinese"
        elif has_simp and has_trad:
            return 'mixed-chinese'

    # English language
    elif language.startswith("en"):
        return 'english'
    else:
        return language

In [16]:
def extract_source(url):
    parsed = urlparse(url)
    domain = parsed.netloc
    return domain

In [ ]:
def parse_json(data, url, id):
    # Parse JSON
    result_json = {
        'id': str(id).zfill(5),
        'questions': None,
        'raw_text': data.get('raw_text'),
        'text': data.get('text'),
        'source': extract_source(url),
        'title': data.get('title'),
        'author': data.get('author'),
        'post_date': data.get('date'),
        'language': check_language(data.get('text')),
        'created_at': data.get('filedate'),
        'excerpt': data.get('excerpt'),
        'tags': [data.get('tags')],
        'link': url
    }
    return result_json

In [18]:
def main():
    id = 13640
    json_list = []
    urls = extract_YI_XIANG_HAO_JU_link()
    print(f'url collection succeed, collectoed {len(urls)} urls')
    seen = set()
    for url in urls:
        if url in seen:
            continue
        seen.add(url)
        data_json = load_web(url)
        time.sleep(2)
        if not data_json:
            print('No result fetched\n')
            continue

        id += 1
        result_json = parse_json(data_json, url, id)
        json_list.append(result_json)
        print('Successfully fetched\n')
    
    print('fetch finished')
    with open("../data/YI_XIANG_HAO_JU.json", "w", encoding="utf-8") as f:
        json.dump(json_list, f, ensure_ascii=False, indent=2)

In [19]:
if __name__ == '__main__':
    main()

accessing https://www.uhouzz.com/news/tags/aozhouzufang.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_1.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_2.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_3.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_4.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_5.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_6.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_7.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_8.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_9.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_10.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_11.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_12.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_13.html
accessing https://www.uhouzz.com/news/tags/aozhouzufang_14.html
accessing https://www.uhouzz.com/news/tags/aozhouzuf